<a href="https://colab.research.google.com/github/MonikaGost/Winc/blob/master/AssignmentModule6_Income_inequality_in_relation_to_GDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

url_gdp = "https://raw.githubusercontent.com/MonikaGost/Winc/master/Assignment%20module%206/national-gdp-constant-usd-wb.csv"
url_ineq = "https://raw.githubusercontent.com/MonikaGost/Winc/master/Assignment%20module%206/inequality.csv"

df_g = pd.read_csv(url_gdp)
df_i = pd.read_csv(url_ineq)

pd.options.display.float_format = '{:.2f}'.format

Inspecting and cleaninig GDP data frame.

In [ ]:
#display(df_g)
#df_g.shape
df_g.drop(columns=["Code"],inplace=True) # dropping unnecessary column
df_g.rename(columns={"GDP (constant 2015 US$)":"GDP","Entity":"Country"}, inplace=True)
df_g.isnull().sum() # no default null values in the df
df_g.dtypes # year is of type int64, at this moment I check first another df before i change something. But for "Country" I will change the type to string now.
display(df_g)

Some functions to check the columns

In [3]:
def cast_to_type(df,column_name, maybe_type):
  print("* Trying to cast to assumed type:")
  try:
    df[column_name].astype(maybe_type,errors="ignore")
    print(f"Casting to {maybe_type} was successful\n")
  except ValueError as error:
    print(f"Could not cast to {maybe_type}: {error}\n")

def unique_values(df,column_name):
  unique_values = df[column_name].unique()
  print(f'* Number of unique values in column \'{column_name}\': {len(unique_values)}\n')
  if len(unique_values)<155:
      print(f'* List of unique values in column \'{column_name}\':')
      try:
        sorted = np.sort(unique_values)
        print(" - Values are sorted")
        display(list(sorted))
      except:
        print(" ! Could not sort values !")
        display(list(unique_values))

def defnullv(df, column_name):
    nulls = df.loc[df[column_name].isnull()]
    print(f"* Number of default null values in column \"{column_name}\": {len(nulls)}\n")
    if len(nulls)<=15 and len(nulls)!=0:
      print(f"* View on the table filtered by Null values in \"{column_name}\":")
      print(nulls)
    elif len(nulls)==0:
      print("There are no default missing values found in this column.")

def column_check(df,column_name,maybe_type):
    column = df[column_name]
    print(f"*** Inspecting column \'{column_name}\' *** \n")
    cast_to_type(df,column_name,maybe_type)
    unique_values(df,column_name)
    print("* Looking at frequency: ")
    display(df[column_name].value_counts(dropna=False))
    print("\n")
    defnullv(df,column_name)

In [ ]:
column_check(df_g,"Country",str)

In [ ]:
column_check(df_g,"Year","datetime64")

Inspecting and cleaning Gini coefficient data frame

In [ ]:
df_i
# I think i need only 3 columns from this one
dfi_2 = df_i.loc[:,"Country":"Gini coefficient (before tax) (World Inequality Database)"]
#display(dfi_2)
dfi_2.rename(columns={"Gini coefficient (before tax) (World Inequality Database)":"Gini_co"}, inplace=True)
display(dfi_2)
dfi_2.dtypes
dfi_2.isnull().sum() # a lot of null values in Gini_coef. but for the moment i leave it like that

In [ ]:
column_check(dfi_2,"Country",str)

In [ ]:
column_check(dfi_2,"Year","datetime64")
# GDP dataframe data starts from 1960, so i will remove older records from this df as we can't compare these years.


In [ ]:
dfi_3 = dfi_2.loc[dfi_2["Year"] >= 1960]
dfi_3
column_check(dfi_3,"Year","datetime64")

Merging the data frames + formatting and cleaning new data frame

In [ ]:
df = pd.merge(df_g, dfi_3, how='left', on=['Country', 'Year'])
display(df)
df.tail(30)


In [ ]:
# Checking if merge worked:
x = df_g[df_g["Country"] == "Afghanistan"]
y = dfi_3[dfi_3["Country"] == "Afghanistan"]
z = df[df["Country"]=="Afghanistan"]

display(x,y,z) # seems it worked well. Still we have some missing data.

In [ ]:
df.shape
df.dtypes

In [ ]:
#Changing "Year" dtype to datetime
from pandas.core.tools.datetimes import to_datetime
df["Year"] = pd.to_datetime(df["Year"], format = "%Y")

df.dtypes

In [ ]:
#I set is as an index:
df.set_index('Year',inplace = True)

display(df)

Calculating correlation cbetween GDP and Gini_coefficient

In [ ]:
# Now i would like to caluculate the correlation coefficient but I see some missing data.
# At first i was thinking to replace them with (for example) mean or based on neighbour values but this would implicit that there is a corelation of any kind,
#so I decided to remove missing values.
df.dropna(inplace=True)
df.shape # from 10669 rows 2390 are left
display(df)
column_check(df,"Country",str)

In [127]:
df1= df.groupby("Country")

In [ ]:
# Calculating correlation coefficient
df1.corr(numeric_only = True) # It looks the corelation between GDP and Gini_co varies per country, it can also have to do with number of data we have as for some
#of the countries there are only few records.
# But in general corelation looks to be quite strong (from realtively strong to very strong ).

# I look on average:
df1.corr(numeric_only = True).mean() # on the average we can say  tha corelation is relatively strong

In [ ]:
# This seemed to me the way to calculate coefficient in this df. I did not use resample here as I wanted to look per country, i think the year does not matter here,
# not sure if this is a proper approach.
# I am very courious about the feedback!